# 第11次实验实验报告

数据的统计与分析

计算机系 计15 2021010803 郭高旭

## 第5题：炮弹问题

### 问题分析与算法设计

二元正态分布的概率密度函数（PDF）通常写作如下形式：

$$ f(x, y) = \frac{1}{2\pi \sigma_x \sigma_y \sqrt{1-\rho^2}} \exp\left( -\frac{1}{2(1-\rho^2)} \left[ \frac{(x-\mu_x)^2}{\sigma_x^2} - 2\rho \frac{(x-\mu_x)(y-\mu_y)}{\sigma_x \sigma_y} + \frac{(y-\mu_y)^2}{\sigma_y^2} \right] \right) $$

其中：
- $ (x, y) $ 是二元正态分布的变量；
- $ \mu_x $ 和 $ \mu_y $ 是 $ x $ 和 $ y $ 的均值；
- $ \sigma_x $ 和 $ \sigma_y $ 是 $ x $ 和 $ y $ 的标准差；
- $ \rho $ 是 $ x $ 和 $ y $ 之间的相关系数。

这个公式描述了在给定均值、标准差和相关系数的情况下，二元正态分布的概率密度。

本体实际上就是计算二元正态分布的概率密度函数在某个区域上的积分。这里我们采用蒙特卡洛方法来计算积分。

注意题目中的二元正态分布由于x,y是相关的,所以只具备关于原点的对称性,而不具备关于x轴或y轴的对称性,所以这里取乐x轴上的部分计算,然后乘以2即可.

### 代码实现与结果

In [1]:
from sympy import symbols, exp, pi, sqrt, lambdify
import numpy as np
import scipy as sp
from scipy.integrate import dblquad

# 定义符号
x, y = symbols('x y')

# 已知的定值参数
mu_x_val, mu_y_val, sigma_x_val, sigma_y_val, rho_val = 0, 0, 80, 50, 0.4

# 定义二维正态分布的概率密度函数，将参数替换为具体的数值
f_expr = (1 / (2*pi*sigma_x_val*sigma_y_val*sqrt(1-rho_val**2))) * \
    exp(-(1/(2*(1-rho_val**2))) *
        ((x-mu_x_val)**2/sigma_x_val**2 - 2*rho_val*(x-mu_x_val)*(y-mu_y_val)/(sigma_x_val*sigma_y_val) +
         (y-mu_y_val)**2/sigma_y_val**2))

# 使用lambdify将f_expr转换为函数，并提供转换字典
f_func = lambdify((x, y), f_expr, modules='numpy')

def y_outer(x):
    return sqrt(100**2-x**2)

result,error=sp.integrate.dblquad(f_func, -100, 100, 0, y_outer)
print("ans:",2*result)


ans: 0.6979392744869912


### 结果分析
积分正确返回结果,0.6979

### 结论
炮弹罗在目标上的概率为0.6979

### 意义
这个问题实际上是一个概率问题,通过计算概率密度函数在某个区域上的积分,可以得到炮弹落在目标上的概率,这个问题实际上是一个概率问题,通过计算概率密度函数在某个区域上的积分,可以得到炮弹落在目标上的概率.

## 第7题：报童问题

### 问题分析与模型建立
在例题中给出了报童一天的平均利润计算方法.
$$
V(n)=\Sigma_{r=0}^{n-1}[(b-a)r-(a-c)(n-r)]f(r)+\Sigma_{r=n}^{\infty}[(b-a)n]f(r)-an
$$
由于r较大,可以将其看作连续随机变量x
$$
V(n)=\int_0^n [(b-a)x-(a-c)(n-x)]p(x)dx+\int_n^{\infty}[(b-a)n]p(x)dx-an
$$
对其求导得到
$$
V'(n) = A(\frac{2n}{K}-1) + c\int_{0}^n p(x)dx + b\int_{n}^{+\infty} p(x)dx
$$
求二阶导可以得到：
$$
V''(n) = \frac{2A}{K} + (c-b)p(x)
$$
当 $\mu$ 远大于 $\sigma$ 时，我们可以近似地将
$$
\int_{0}^n p(x)dx
$$
看作：
$$
1-\int_{n}^{+\infty} p(x)dx
$$
令一阶导数为0,有:
$$
A(\frac{2n_0}{K}-1) + b + (c-b)\Phi(\frac{n_0-\mu}{\sigma}) = 0
$$

其中 $\Phi$ 表示标准正态分布的累积分布函数。我们需要找到一个 $n_0$，使得：
$$
\Phi(\frac{n_0-\mu}{\sigma}) = \frac{A(\frac{2n_0}{K}-1) + b}{b-c}
$$
这里的$n_0%就是报童应该购入的报纸数量.
### 代码实现与结果

In [2]:
import numpy as np
from scipy.optimize import fsolve
from scipy.stats import norm

def calculate_optimal_nums(mean, std_dev, cost_per_num, fixed_cost, min_profit, max_profit):
    def equation(n):
        return norm.cdf(n, mean, std_dev) - (cost_per_num * (2 * n / fixed_cost - 1) + min_profit) / (min_profit - max_profit)
    return fsolve(equation, mean)[0]

def calculate_second_derivative(mean, std_dev, cost_per_num, fixed_cost, min_profit, max_profit, optimal_nums):
    return 2 * cost_per_num / fixed_cost + (max_profit - min_profit) * norm.pdf(optimal_nums, mean, std_dev)

def main():
    mean, std_dev = 2000, 50
    cost_per_num, fixed_cost = 0.5, 50000
    min_profit, max_profit = 0.5, 0.35

    optimal_nums = calculate_optimal_nums(mean, std_dev, cost_per_num, fixed_cost, min_profit, max_profit)
    second_derivative = calculate_second_derivative(mean, std_dev, cost_per_num, fixed_cost, min_profit, max_profit, optimal_nums)

    print(f"Optimal number of nums: {optimal_nums:.4f}")
    print(f"Second derivative of profit function: {second_derivative:.4f}")

if __name__ == "__main__":
    main()

Optimal number of nums: 1968.2060
Second derivative of profit function: -0.0010


### 结果分析

根据计算结果，我们得到 $n_0 = 1968$，并且 $f''(n_0) = -1.0\times 10^{-3} < 0$。因此，最优解为 $1968$。所以报童购入1968份报纸即为利润最大时的购入量.

### 意义

通过对报童问题的分析,可以使用微积分的方法来解决一些实际问题,通过对问题的建模,可以得到问题的最优解,这对于实际生活中的决策有很大的帮助.

## 第c题：

### 问题分析与模型建立

### 最小化钢材浪费的平均长度算法

我们要确定粗轧机器的平均长度 $ \mu $ 以最小化钢材的浪费。设粗轧后的钢材长度 $ X $ 服从正态分布 $ N(\mu, \sigma^2) $，其中 $ \sigma = 0.2 $。过大的 $ \mu $ 会导致精轧浪费钢材，而过小的 $ \mu $ 则会增加报废的概率。

浪费函数 $ w(x) $ 定义如下：
$$ w(x) = \begin{cases} 
x & \text{if } x < L \\
x - L & \text{if } x \geq L 
\end{cases} $$

浪费的期望值 $ E[w(X)] $ 为：
$$ E[w(X)] = \int_{0}^{L} x P(X = x) \, dx + \int_{L}^{\infty} (x - L) P(X = x) \, dx $$

概率密度函数 $ P(X = x) $ 约为：
$$ P(X = x) \approx \frac{1}{\sqrt{2\pi} \sigma} e^{-\frac{(x - \mu)^2}{2\sigma^2}} $$

接下来我们计算 $ E[w(X)] $：
$$ E[w(X)] = \int_{0}^{\infty} x P(X = x) \, dx - L \int_{L}^{\infty} P(X = x) \, dx \approx \mu - L \cdot P(X > L) $$
$$ E[w(X)] \approx \mu - L \left(1 - \Phi\left(\frac{L - \mu}{\sigma}\right)\right) $$

设 $ P(X > L) \approx 1 - \Phi\left(\frac{L - \mu}{\sigma}\right) $，其中 $ \Phi $ 是标准正态分布的累积分布函数。

为了最小化期望浪费，我们需要最小化以下目标函数：
$$ f(\mu) = \mu - L \left(1 - \Phi\left(\frac{L - \mu}{0.2}\right)\right) $$

### 算法设计

使用数值优化方法（如牛顿法或梯度下降法）来找到使 $ f(\mu) $ 最小化的 $ \mu $ 值，同时满足 $ \mu > 0 $。

```python
import numpy as np
from scipy.optimize import minimize
from scipy.stats import norm

def objective_function(mu, L=1.0, sigma=0.2):
    return mu - L * (1 - norm.cdf((L - mu) / sigma))

L = 1.0
initial_guess = L
result = minimize(objective_function, initial_guess, args=(L, 0.2), bounds=[(0, None)])
optimal_mu = result.x[0]

print(f"Optimal average length μ: {optimal_mu:.4f}")
```

此算法通过数值优化找到最优的 $ \mu $ 值，以最小化钢材浪费。需要考虑实际情况中的更多复杂因素来调整策略。

### 代码实现与结果

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize_scalar
from scipy.stats import norm

def calculate_expected_waste_one_bar(m, l, sigma):
    return m - l * (1 - norm.cdf((l - m) / sigma))

def calculate_expected_waste_per_obtained_bar(m, l, sigma):
    return m / (1 - norm.cdf((l - m) / sigma)) - l

def optimize_function(func, l, sigma, bounds=(0, 4)):
    result = minimize_scalar(func, args=(l, sigma), bounds=bounds, method='bounded')
    return result.x, result.fun

def main():
    l = 2.0
    sigma = 0.2

    m_opt1, fval1 = optimize_function(calculate_expected_waste_one_bar, l, sigma)
    m_opt2, fval2 = optimize_function(calculate_expected_waste_per_obtained_bar, l, sigma)

    print("Optimization Result for Expected Waste for One Steel Bar:")
    print(f"Optimal m: {m_opt1:.4f}")
    print(f"Function Value: {fval1:.4f}")
    print()
    print("Optimization Result for Expected Waste for Each Obtained Steel Bar:")
    print(f"Optimal m: {m_opt2:.4f}")
    print(f"Function Value: {fval2:.4f}")


if __name__ == "__main__":
    main()


Optimization Result for Expected Waste for One Steel Bar:
Optimal m: 2.3327
Function Value: 0.4289

Optimization Result for Expected Waste for Each Obtained Steel Bar:
Optimal m: 2.3562
Function Value: 0.4479


### 结果分析,结论

在粗轧过程中，钢材的最小期望浪费值为 $0.4289$，当 $m = 2.3327$ 时达到最小值。而对于获得一根规定长度的钢材，最小期望浪费值为 $0.4479$，当 $m = 2.3562$ 时达到最小值。

### 意义

通过对钢材浪费的期望值进行分析，我们可以找到最优的粗轧机器平均长度，以最小化钢材的浪费。这对于生产企业来说是非常重要的，可以提高生产效率，降低生产成本。
